In [1]:
from sqlalchemy import create_engine
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
localhost = {'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432, 'db': 'fiadb'}
params = 'postgresql://{0}:{1}@{2}:{3}/{4}'
engine = create_engine(params.format(localhost['user'], localhost['password'], localhost['host'], localhost['port'], localhost['db']))
select_query = """
select a.tsr, a.tsr_predicted as glm_pred,
b.tsr_predicted as rf_pred,
c.tsr_predicted as grnn_pred,
d.tsr_predicted as ffnn_pred
FROM predictor.lasso_glm a, predictor.random_forest b, predictor.grnn c, predictor.ffnn d
where a.grid_id = b.grid_id
and a.grid_id = c.grid_id
and a.grid_id = d.grid_id
"""
pred_results = pd.read_sql(select_query, engine)

In [3]:
pred_results.head()

,tsr,glm_pred,rf_pred,grnn_pred,ffnn_pred
0,4.0,7.248847,2.944604,3.276433,1.84324
1,2.0,7.243437,2.626410,2.000123,2.25253
2,5.0,7.086875,3.855186,4.994345,4.10530
3,1.0,6.216415,1.820770,2.803938,1.72234
4,1.0,6.471408,2.102745,2.795763,1.84148


In [31]:
pred_results.shape

(15310, 5)

In [25]:
from scipy.stats import wilcoxon
from statistics import median

In [21]:
glm_w, glm_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['glm_pred'].tolist())

(56665718.0, 0.00039688720450696893)

In [22]:
rf_w, rf_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['rf_pred'].tolist())

(57574340.0, 0.06001473650742681)

In [23]:
grnn_w, grnn_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['grnn_pred'].tolist())

(57483148.0, 0.0406154561673107)

In [24]:
ffnn_w, ffnn_p = wilcoxon(pred_results['tsr'].tolist(), pred_results['ffnn_pred'].tolist())

(55302952.5, 1.5992205163465825e-09)

In [28]:
print(median(pred_results['tsr'].tolist()), median(pred_results['glm_pred'].tolist()),
     median(pred_results['rf_pred'].tolist()), median(pred_results['grnn_pred'].tolist()),
     median(pred_results['ffnn_pred'].tolist()))

12.0 13.5228224656636 12.5873510233424 12.71703548197955 12.381


In [29]:
print(glm_w, rf_w, grnn_w, ffnn_w)

56665718.0 57574340.0 57483148.0 55302952.5


In [30]:
print(glm_p, rf_p, grnn_p, ffnn_p)

0.00039688720450696893 0.06001473650742681 0.0406154561673107 1.5992205163465825e-09
